將all_dataset.csv轉換成DB，表格badminton_data

In [8]:
import pandas as pd
import sqlite3

# === 1. 讀取 CSV 檔案 ===
csv_file = "all_dataset.csv"   # 你的 CSV 檔名
df = pd.read_csv(csv_file)

# === 2. 建立或連線 SQLite 資料庫 (會建立 data.db 檔案) ===
db_file = "data.db"
conn = sqlite3.connect(db_file)

# === 3. 將 CSV 存成一張表格 ===
table_name = "badminton_data"   # 自訂表格名稱
df.to_sql(table_name, conn, if_exists="replace", index=False)

print(f"✅ CSV 已成功存入 SQLite 資料庫 {db_file} 的 {table_name} 表格")

# === 4. 測試讀取前 5 筆資料 ===
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
rows = cursor.fetchall()

print("前 5 筆資料：")
for row in rows:
    print(row)

# === 5. 關閉連線 ===
conn.close()


✅ CSV 已成功存入 SQLite 資料庫 data.db 的 badminton_data 表格
前 5 筆資料：
(1, 1, '0:07:39', 11496.0, 0.0, 0.0, 0.0, 'Kento MOMOTA', 1.0, '發短球', 0.0, 1.0, 2.0, 14.0, -0.0196492380571161, 0.3664146155198778, 1.0, 18.0, -0.073337499654345, 0.5945635355250196, None, None, 'Kento MOMOTA', None, 14.0, -0.0196492380571161, 0.3664146155198778, 0.0316904856227849, -0.1035456857115604, 10.0, -0.1632827887015297, 0.285619010735518, 0.0, 0.0, 0.0, 1.0, 0.0, 11.0, 0.0120412475656688, 0.2628689298083174, None, 0.0, 0.0, 0.0, 1, 1.0, 0)
(1, 2, '00:07:43', 11582.0, 0.0, 0.0, 0.0, 'CHOU Tien Chen', 3.0, '長球', 0.0, 1.0, 2.0, 18.0, -0.073337499654345, 0.5945635355250196, 2.0, 32.0, -0.4513914856846306, -0.4228076653977647, '出界', '對手出界', 'Kento MOMOTA', None, 10.0, -0.1632827887015297, 0.285619010735518, 0.0, 0.0, 11.0, 0.0120412475656688, 0.2628689298083174, 0.0316904856227849, -0.1035456857115604, 0.3217714630280202, None, None, 18.0, -0.073337499654345, 0.5945635355250196, None, 0.0899452890471847, 0.308944524789501

In [ ]:
pip install -U bitsandbytes rank_bm25

In [ ]:
pip install "accelerate>=0.26.0"

In [ ]:
pip install transformers

In [ ]:
pip install jieba

In [ ]:
pip install tf-keras

In [ ]:
pip install sentence_transformers

In [ ]:
pip install accelerate

In [4]:
# 載入所需的套件（Packages）
# Import necessary libraries

import json                         # 處理 JSON 格式資料的標準套件 / Standard JSON library
import torch                        # PyTorch 是一個深度學習框架 / PyTorch deep learning framework
from typing import List             # 用來註解 List 型別 / Type annotation for list
from transformers import (          # Transformers 模型相關函式庫 / HuggingFace Transformers library
    AutoModelForCausalLM,           # 自動載入因果語言模型 / Auto loader for causal language models
    AutoTokenizer,                  # 自動載入對應的 tokenizer / Auto loader for tokenizer
    BitsAndBytesConfig,             # 用來設定量化模型的參數 / Configuration for model quantization
    pipeline                        # 提供簡單的模型推論介面 / Easy interface for model inference
)
from tqdm.autonotebook import tqdm  # 顯示進度條（適用於 Jupyter Notebook） / Progress bar in Jupyter
import jieba                        # 中文斷詞工具 / Chinese word segmentation
from rank_bm25 import BM25Okapi     # BM25 搜尋模型 / BM25 retrieval model
from sentence_transformers import SentenceTransformer  # 向量模型 / Sentence embedding model

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# ====== 設定量化參數，減少記憶體使用 / Set quantization settings for smaller memory usage ======
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # 使用 4-bit 權重 / Use 4-bit weights
    bnb_4bit_use_double_quant=True,       # 啟用雙重量化 / Enable double quantization
    bnb_4bit_quant_type="nf4",            # 使用 nf4 量化類型 / Quantization type
    bnb_4bit_compute_dtype=torch.bfloat16 # 使用 bfloat16 進行計算 / Use bfloat16 for compute
)

# ====== 檢查 GPU ======
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用裝置: {device}")

# ====== 載入微調好的語言模型（Gemma） / Load pretrained LLM ======
llm = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="unsloth/gemma-3-4b-it",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",             # 自動分配模型到 GPU（支援多 GPU）
    low_cpu_mem_usage=True,
)

# ====== 載入對應的 tokenizer / Load tokenizer for the model ======
tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-3-4b-it")

# ====== 使用 pipeline 包裝模型推論介面 / Create a pipeline for generation ======
llm_pipe = pipeline(
    "text-generation",            # 任務為文本生成 / Task type
    model=llm,                    # 使用的模型 / LLM
    tokenizer=tokenizer,          # tokenizer
    max_new_tokens=512,           # 回應最大長度 / Maximum new tokens
    do_sample=False,              # 不使用隨機 sampling（使用貪婪解碼）/ Greedy decoding
    temperature=None,
    top_p=None,
    top_k=None,
    device_map="auto",            # 自動使用 GPU / Automatically use GPU
)

# ====== 測試生成 ======
prompt = "介紹羽球"
outputs = llm_pipe(prompt)
print(outputs[0]["generated_text"])


使用裝置: cuda


`torch_dtype` is deprecated! Use `dtype` instead!
Exception in thread Thread-13 (_readerthread):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\envs\AI\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\anaconda3\envs\AI\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\ProgramData\anaconda3\envs\AI\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 7: invalid start byte


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.


介紹羽球賽事的重要細節，包含對手的詳細紹介、比賽的流程安排、以及比賽的規則及重要規則變動，並提供賽後分析，總結比賽結果及對下賽事的影響。

**內容概要：**

1.  **對手介紹：**
    *   對手基本資料（姓名、國籍、球齡、職業排名、擅長球種、特點）：例如，"張小明，來自中國，球齡10年，世界排名第200名，擅長發球，對速度型球較敏感"。
    *   對手近期表現：近期比賽成績、優劣勢分析、賽前狀態評估。例如，"上週在上海比賽，以三分兩負晉級，雖然狀態不錯，但對左手位偏弱。"
    *   對手賽事風格及弱點：了解對手的打法習慣、擅長與不擅長，以及可以利用的弱點。例如，"對變陣球反應較慢，容易被快攻打亂節奏"。

2.  **比賽流程：**
    *   比賽時間、地點、場地狀況：天氣、場地材質、光線等影響因素。
    *   比賽形式（單打、雙打、團體）：明顯指出比賽種類。
    *   比賽分局及總局：說明每局的得分方式（21分制，先11分勝一局，但局數的得分必須是奇數），以及勝利條件。
    *   比賽時間安排（每局的時間限制、休息時間）。

3.  **比賽規則：**
    *   基本規則：得分方式、發球規則、服務規則、破發規則、恢復球規則、禁區規則等。
    *   重要規則變動：賽前是否有針對特定規則進行調整？例如，賽前是否允許使用膠墊？
    *   特殊規則：針對特殊情況的規則，例如：補賽規則、四局半規則。

4.  **規則及重要規則變動細節：**
    *   詳細解釋每個規則的細節和應用場景。例如：發球時，必須在服務線上方發球，且球必須落在對手服務區。
    *   說明規則變動對比賽的影響。例如：允許使用膠墊，可能會


使用RAG給定預先設計好的簡單問題及對應代碼

可能要想想看
user問題: 周天成在第一局用了幾個殺球?
這種附有人名的該如何撰寫問題集??
還是應該這樣處理
問題集: 某人在第一局用了幾個殺球?
匹配到後，再把人名填進去?(使用LLM幫忙填嗎?還是程式碼判斷填入?)

In [6]:
import json
from typing import List

# 讀取 badminton.json
with open("badminton_query_templates.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 將 question 當作 chunk
chunks = [item["question"] for item in data]
sql_template = [item["sql_template"] for item in data]


In [7]:
import jieba
from rank_bm25 import BM25Okapi

# 對每個 chunk 斷詞
tokenized_chunks = [list(jieba.cut(chunk)) for chunk in chunks]

# 建立 BM25 模型
bm25 = BM25Okapi(tokenized_chunks)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Danie\AppData\Local\Temp\jieba.cache
Loading model cost 0.422 seconds.
Prefix dict has been built successfully.


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 載入多語言 embedding 模型
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

# 計算每個 chunk 的向量
chunks_embeddings = embedding_model.encode(chunks, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
def bm25_retrieve(query: str, chunks: List[str], bm25: BM25Okapi) -> List[str]:
    tokenized_query = list(jieba.cut(query))
    scores = bm25.get_scores(tokenized_query)
    rank = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)
    return [chunk for chunk, _ in rank]

def embedding_retrieve(query: str, chunks: List[str], chunks_embeddings: np.ndarray, embedding_model: SentenceTransformer) -> List[str]:
    query_embedding = embedding_model.encode(query)
    # 計算餘弦相似度
    scores = np.dot(chunks_embeddings, query_embedding) / (np.linalg.norm(chunks_embeddings, axis=1) * np.linalg.norm(query_embedding))
    rank = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)
    return [chunk for chunk, _ in rank]

def reciprocal_rank_fusion(*ranked_lists, k=60) -> List[str]:
    scores = {}
    for rl in ranked_lists:
        for rank, doc in enumerate(rl, start=1):
            scores[doc] = scores.get(doc, 0.0) + 1.0 / (k + rank)
    fused = sorted(scores.items(), key=lambda x: -x[1])
    return [doc for doc, _ in fused]



In [10]:
#query = "這位選手大多站在哪裡發球？"
query = input("請輸入您的查詢問題：")

# 各檢索結果
bm25_ranked = bm25_retrieve(query, chunks, bm25)
embedding_ranked = embedding_retrieve(query, chunks, chunks_embeddings, embedding_model)

# 融合
fused_ranked = reciprocal_rank_fusion(bm25_ranked, embedding_ranked)

print("Top 1 similar:", fused_ranked[0])

top1 = fused_ranked[0]
top1_idx = chunks.index(top1)
similarity_score = np.dot(chunks_embeddings[top1_idx], embedding_model.encode(query)) / \
                   (np.linalg.norm(chunks_embeddings[top1_idx]) * np.linalg.norm(embedding_model.encode(query)))

if similarity_score < 0.95:
    print("無匹配")
else:
    print("匹配到類別:", sql_template[top1_idx])
    print(similarity_score)



Top 1 similar: 這位選手在多數回合中主要站在哪個場區發球或接球？
匹配到類別: SELECT player_location_area, COUNT(*) AS freq FROM all_dataset WHERE server = '{player_name}' GROUP BY player_location_area ORDER BY freq DESC LIMIT 1;
0.952892


若不匹配詢問小型LLM是否可以只用SELECT完成撰寫程式碼

In [11]:
import json

# === 1️⃣ 定義 JSON 檔案名稱 ===
json_file = "column_definition.json" # 你的 JSON 檔名

# === 2️⃣ 載入欄位定義 JSON 檔 ===
with open(json_file, "r", encoding="utf-8") as f:
    full_definitions = json.load(f)

# 從 JSON 檔中取得 'data_columns' 部分
column_definitions = full_definitions.get("data_columns", [])

# Add a print statement to inspect the structure
#print("--- 載入的 JSON 內容類型 ---")
#print(type(full_definitions))
#print("\n--- 'data_columns' 內容 ---")
#print(type(column_definitions))
#print(column_definitions)

# === 3️⃣ 將欄位定義轉成文字說明 ===
# 確保 column_definitions 是一個列表，且每個元素都有 'column' 和 'definition' 鍵
if isinstance(column_definitions, list) and all(isinstance(item, dict) and 'column' in item and 'definition' in item for item in column_definitions):
    column_info_text = "\n".join(
        [f"- {item['column']}: {item['definition']}" for item in column_definitions]
    )
else:
    column_info_text = "錯誤：'data_columns' 的格式不符合預期。"


# --- 定義 System Prompt ---

EXTENDED_SYSTEM_PROMPT = f"""
You are an expert in badminton sports and data analysis.
Below are the column names and their definitions for the data table. Please refer to these definitions when answering questions:

{column_info_text}

If the question cannot be answered using a single simple SELECT statement,
respond exactly with: "I can not help you." Otherwise, please write SQL code for the user to query in SQLite, avoiding instructions unrelated to the question.
**important** Only provide SQL code
""".strip()

# --- 定義使用者 Prompt 模板 ---
USER_PROMPT = """
### Question: 
{query}
""".strip()

# --- 假設使用者輸入 ---
#query = "查詢第一局第一回合的最終比數"
#query = "告訴我CHOU Tien Chen在第一局使用幾個殺球"
#query = input("請輸入您的查詢問題：")

# --- 建立聊天格式 ---
chats = [
    {"role": "system", "content": EXTENDED_SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT.format(query=query)}
]

# --- 呼叫小模型（Gemma-3-4B-it） ---
response = llm_pipe(chats)[0]['generated_text'][-1]['content'].strip()

# --- 輸出結果 ---
print("🧩 LLM 輸出：")
print(response)


🧩 LLM 輸出：
```sqlite
SELECT player_location_area FROM badminton_data WHERE player = '選手姓名'
```
